In [1]:
import get_two_circle_datasets as gtcd
import numpy as np
import matplotlib.pyplot as plt
from ripser import Rips
import spectrum_distance_sup_operator_l1 as sd
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import fowlkes_mallows_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import AffinityPropagation

In [2]:
def conformal_linear_transformation_point_cloud(points, angle_range=(0, 360)):
    angle = np.random.uniform(*angle_range)
    angle = angle * np.pi/180
    random_vector = np.random.rand(1, 2)
    random_scale = np.random.uniform(0.1,10)
    cos = np.cos(angle)
    sin = np.sin(angle)
    R = np.array([[cos, -sin], [sin, cos]])
    P_rotated = points.dot(R)
    P_rotated = random_scale * P_rotated + random_vector
    return P_rotated

In [3]:
"""
class1 three elliptics
class2 two elliptics
"""
a, b = 40, 35  
a_s, b_s = 20, 12.5
center = (0, 0)
center_1 = (0,80)
num_before_transform = 150
N_in_class = 200
N = 300
N_per_class = int(N / 2)
half = int(N_in_class / 2)
per_two_loop_num = 240
per_two_loop_num_half = 120

In [4]:
%%time
from tqdm import tqdm
mean_acl_num_cluster = 0
mean_ap_num_cluster = 0
mean_acl_score = 0
mean_ap_score = 0
mean_km_score = 0
for i in tqdm(range(100), desc='Spectrum'):
    """
    get three elliptics
    """
    three_elliptics= []
    for _ in range(num_before_transform):
        add_ellip = gtcd.get_random_small_ellipse_add_noise(35,25, center_1, 40)
        te = gtcd.get_random_two_ellipse(a, b, a_s, b_s, center, half)
        te = np.append(te,add_ellip,axis = 0)
        rotate_te = conformal_linear_transformation_point_cloud(te)
        three_elliptics.append(rotate_te)

    """
    class 1: three elliptics
    """
    class_1 = three_elliptics
    
    """
    class_2:two elliptics
    """
    just_elliptic = [conformal_linear_transformation_point_cloud(gtcd.get_random_two_ellipse(a, b, a_s, b_s, center, per_two_loop_num_half)) for _ in range(num_before_transform)]

    class_2 = just_elliptic
    #print(len(class_2))

    datas = []
    datas.extend(class_1)
    datas.extend(class_2)
    labels = np.zeros(N)
    labels[N_per_class:] = 1

    """
    rips
    """
    rips = Rips(maxdim=1, coeff=2)

    diagrams_h1_delete_inf = gtcd.rips_delete_inf(datas, 1)
    
    """
    generate distance matrix
    """

    def get_sc_dismatrix(datas):
        distance_matrix = np.zeros((len(datas), len(datas)))
        #print("distance_matrix.shape", distance_matrix.shape)
        for i in range(len(datas)):
            for j in range(len(datas)):
                distance_matrix[i, j] = sd.compute_distance(datas[i], datas[j])
        return distance_matrix

    A = get_sc_dismatrix(diagrams_h1_delete_inf)
    
    """
    Agglomerative Clustering
    """
    

    acl = AgglomerativeClustering(n_clusters=2, metric='precomputed', linkage='average')

    acl_labels = acl.fit_predict(A)
    
    #mean_acl_num_cluster += len(set(acl_labels))


    #from sklearn.metrics import fowlkes_mallows_score
    acl_score = fowlkes_mallows_score(labels, acl_labels)
    print("acl_score",acl_score)
    mean_acl_score += acl_score

    """
    Affinity Propagation
    """
    

    ap = AffinityPropagation(affinity='precomputed')
    ap_clustering = ap.fit(A)
    ap_labels = ap_clustering.labels_
    
    mean_ap_num_cluster+=len(set(ap_labels))

    from sklearn.metrics import fowlkes_mallows_score

    ap_score = fowlkes_mallows_score(labels, ap_labels)
    print("ap_score",ap_score)
    mean_ap_score += ap_score
    
    """
    k-med
    """
    kmedoids = KMedoids(n_clusters=2, random_state=0).fit(A)
    #print(kmedoids.labels_)
    km_score = fowlkes_mallows_score(labels, kmedoids.labels_)
    print("km_score",km_score)
    mean_km_score+=km_score
    

#average_acl_cluster = mean_acl_num_cluster/100
average_acl_score = mean_acl_score/100
average_ap_cluster = mean_ap_num_cluster/100
average_ap_score = mean_ap_score/100
average_km_score = mean_km_score/100
#print("average Agglomerative cluster",average_acl_cluster)
print(f'FMI_Agglomerative= {average_acl_score:.3f}')
print("average Affinity Propagation cluster",average_ap_cluster)
print(f'FMI_Affinity_Propagation = {average_ap_score:.3f}')
print(f'FMI_k_medoids = {average_km_score:.3f}')

Spectrum:   0%|                                                                                | 0/100 [00:00<?, ?it/s]

Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)
Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


Spectrum:   1%|▋                                                                     | 1/100 [00:54<1:29:56, 54.52s/it]

acl_score 0.7067305881955918
ap_score 0.7349486388630451
km_score 0.8815444570386363
Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)
Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


Spectrum:   2%|█▍                                                                    | 2/100 [01:49<1:29:12, 54.61s/it]

...

acl_score 1.0
ap_score 0.7278012677496369
km_score 0.898937966737021
Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)
Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


Spectrum:  99%|████████████████████████████████████████████████████████████████████▎| 99/100 [1:30:52<00:55, 55.47s/it]

acl_score 0.767340385543314
ap_score 0.738630948137842
km_score 0.8537664163172938
Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)
Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


Spectrum: 100%|████████████████████████████████████████████████████████████████████| 100/100 [1:31:47<00:00, 55.07s/it]

acl_score 0.9866678636490931
ap_score 0.766418889502014
km_score 0.8815444570386363
FMI_Agglomerative= 0.844
average Affinity Propagation cluster 2.07
FMI_Affinity_Propagation = 0.721
FMI_k_medoids = 0.852
CPU times: total: 3h 38min 33s
Wall time: 1h 31min 47s
